In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import nltk
from nltk.corpus import treebank
from nltk.corpus import brown

In [ ]:
# import seaborn as sns

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
# from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Input, Activation
from keras.layers import TimeDistributed, RepeatVector
from keras.layers import LSTM, GRU, Bidirectional, SimpleRNN, RNN
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.text import Tokenizer

In [ ]:
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [ ]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [ ]:
brown_corpus = brown.tagged_sents(tagset='universal')

In [ ]:
tagged_sentences = brown_corpus

In [ ]:
X_sentences = []
Y_tags = []
for item in tagged_sentences:
  X = []
  Y = []
  for e in item:
    X.append(e[0])
    Y.append(e[1])
  X_sentences.append(X)
  Y_tags.append(Y)

In [ ]:
X_sentences[1], Y_tags[1]

(['The',
  'jury',
  'further',
  'said',
  'in',
  'term-end',
  'presentments',
  'that',
  'the',
  'City',
  'Executive',
  'Committee',
  ',',
  'which',
  'had',
  'over-all',
  'charge',
  'of',
  'the',
  'election',
  ',',
  '``',
  'deserves',
  'the',
  'praise',
  'and',
  'thanks',
  'of',
  'the',
  'City',
  'of',
  'Atlanta',
  "''",
  'for',
  'the',
  'manner',
  'in',
  'which',
  'the',
  'election',
  'was',
  'conducted',
  '.'],
 ['DET',
  'NOUN',
  'ADV',
  'VERB',
  'ADP',
  'NOUN',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  'ADJ',
  'NOUN',
  '.',
  'DET',
  'VERB',
  'ADJ',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  '.',
  '.',
  'VERB',
  'DET',
  'NOUN',
  'CONJ',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  'ADP',
  'NOUN',
  '.',
  'ADP',
  'DET',
  'NOUN',
  'ADP',
  'DET',
  'DET',
  'NOUN',
  'VERB',
  'VERB',
  '.'])

In [ ]:
print(len(X_sentences))

57340


In [ ]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(X_sentences)
X_encoded = word_tokenizer.texts_to_sequences(X_sentences)

In [ ]:
print((word_tokenizer.word_index))

{'the': 1, ',': 2, '.': 3, 'of': 4, 'and': 5, 'to': 6, 'a': 7, 'in': 8, 'that': 9, 'is': 10, 'was': 11, 'he': 12, 'for': 13, '``': 14, "''": 15, 'it': 16, 'with': 17, 'as': 18, 'his': 19, 'on': 20, 'be': 21, ';': 22, 'at': 23, 'by': 24, 'i': 25, 'this': 26, 'had': 27, '?': 28, 'not': 29, 'are': 30, 'but': 31, 'from': 32, 'or': 33, 'have': 34, 'an': 35, 'they': 36, 'which': 37, '--': 38, 'one': 39, 'you': 40, 'were': 41, 'her': 42, 'all': 43, 'she': 44, 'there': 45, 'would': 46, 'their': 47, 'we': 48, 'him': 49, 'been': 50, ')': 51, 'has': 52, '(': 53, 'when': 54, 'who': 55, 'will': 56, 'more': 57, 'if': 58, 'no': 59, 'out': 60, 'so': 61, 'said': 62, 'what': 63, 'up': 64, 'its': 65, 'about': 66, ':': 67, 'into': 68, 'than': 69, 'them': 70, 'can': 71, 'only': 72, 'other': 73, 'new': 74, 'some': 75, 'could': 76, 'time': 77, '!': 78, 'these': 79, 'two': 80, 'may': 81, 'then': 82, 'do': 83, 'first': 84, 'any': 85, 'my': 86, 'now': 87, 'such': 88, 'like': 89, 'our': 90, 'over': 91, 'man': 92

In [ ]:
tag_tokenizer = Tokenizer()
tag_tokenizer.fit_on_texts(Y_tags)
Y_encoded = tag_tokenizer.texts_to_sequences(Y_tags)

In [ ]:
from keras.utils import pad_sequences

In [ ]:
X_padded = pad_sequences(X_encoded, maxlen=90, padding="pre", truncating="post") #X_padded = pad_sequences(X_encoded, maxlen=150, padding="pre", truncating="post")
Y_padded = pad_sequences(Y_encoded, maxlen=90, padding="pre", truncating="post")

In [ ]:
X, Y = X_padded, Y_padded

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/GoogleNews-vectors-negative300.bin'
word2vec = KeyedVectors.load_word2vec_format(path, binary=True)

FileNotFoundError: ignored

In [ ]:

embedding_weights = np.zeros((len(word_tokenizer.word_index) + 1, 300))
word_dict = word_tokenizer.word_index

for word, index in word_dict.items():
    try:
        embedding_weights[index, :] = word_dict[word]
    except KeyError:
        pass

In [ ]:
print(embedding_weights.shape)

(49816, 300)


In [ ]:
Y = to_categorical(Y)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=4)

In [ ]:
print("TRAINING DATA")
print('Shape of input sequences: {}'.format(X_train.shape))
print('Shape of output sequences: {}'.format(Y_train.shape))
print("-"*50)
print("TESTING DATA")
print('Shape of input sequences: {}'.format(X_test.shape))
print('Shape of output sequences: {}'.format(Y_test.shape))

TRAINING DATA
Shape of input sequences: (45872, 150)
Shape of output sequences: (45872, 150, 13)
--------------------------------------------------
TESTING DATA
Shape of input sequences: (11468, 150)
Shape of output sequences: (11468, 150, 13)


## FFNN Model

In [ ]:
nn_model = Sequential()
nn_model.add(Embedding(input_dim     =  len(word_dict) + 1,
                        output_dim    =  300,
                        input_length  =  90,
                        weights = [embedding_weights],
                        trainable     =  True
))


nn_model.add(Dense(256, input_dim=300))
nn_model.add(Activation('relu'))
nn_model.add(Dense(13, activation="softmax"))
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
nn_model.compile(loss      =  'categorical_crossentropy',
                  optimizer =  sgd,
                  metrics   =  ['accuracy'])



/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 150, 300)          14944800  
                                                                 
 dense_2 (Dense)             (None, 150, 256)          77056     
                                                                 
 activation_1 (Activation)   (None, 150, 256)          0         
                                                                 
 dense_3 (Dense)             (None, 150, 13)           3341      
                                                                 
Total params: 15,025,197
Trainable params: 15,025,197
Non-trainable params: 0
_________________________________________________________________


In [ ]:
nn_training = nn_model.fit(X_train, Y_train, batch_size=128, epochs=10)

Epoch 1/10
359/359 [==============================] - 87s 241ms/step - loss: 928694.5000 - accuracy: 0.8579
Epoch 2/10
359/359 [==============================] - 85s 236ms/step - loss: 0.7020 - accuracy: 0.8650
Epoch 3/10
359/359 [==============================] - 85s 235ms/step - loss: 0.6952 - accuracy: 0.8650
Epoch 4/10
359/359 [==============================] - 86s 240ms/step - loss: 0.6920 - accuracy: 0.8650
Epoch 5/10
359/359 [==============================] - 85s 236ms/step - loss: 0.6902 - accuracy: 0.8650
Epoch 6/10
359/359 [==============================] - 82s 228ms/step - loss: 0.6891 - accuracy: 0.8650
Epoch 7/10
359/359 [==============================] - 85s 237ms/step - loss: 0.6884 - accuracy: 0.8650
Epoch 8/10
359/359 [==============================] - 85s 238ms/step - loss: 0.6879 - accuracy: 0.8650
Epoch 9/10
359/359 [==============================] - 83s 232ms/step - loss: 0.6875 - accuracy: 0.8650
Epoch 10/10
359/359 [==============================] - 86s 239ms/ste

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print("Model Accuracy on test set = ", nn_model.evaluate(X_test,Y_test))

359/359 [==============================] - 7s 20ms/step - loss: 0.6882 - accuracy: 0.8649
Model Accuracy on test set =  [0.6881511807441711, 0.8648691773414612]


## Encoder-Decoder Model

In [ ]:
# encoder_input = Input(shape=(150,))
# encoder_embedded = Embedding(input_dim=len(word_dict)+1, output_dim=300, weights=[embedding_weights], trainable = True)(
#     encoder_input
# )

# # Return states in addition to output
# output, state_h, state_c = LSTM(64, return_state=True, name="encoder")(
#     encoder_embedded
# )
# encoder_state = [state_h, state_c]

# # decoder_input = Input(shape=(None,))
# # decoder_embedded = Embedding(input_dim=, output_dim=64)(
# #     decoder_input
# # )

# # Pass the 2 states to a new LSTM layer, as initial state
# decoder_output = LSTM(64, name="decoder")(
#     decoder_embedded, initial_state=encoder_state
# )
# output = Dense(150,activation="softmax")(decoder_output)

# ed_model = Model(encoder_input, output)
# ed_model.summary()
input_sequence = Input(shape=(150,))
embedding = Embedding(input_dim=len(word_dict)+1, output_dim=300, weights=[embedding_weights], trainable=True)(input_sequence)
encoder = LSTM(64, return_sequences=False)(embedding)
r_vec = RepeatVector(150)(encoder)
decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(13))(decoder)
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
enc_dec_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 300)          14944800  
                                                                 
 lstm (LSTM)                 (None, 64)                93440     
                                                                 
 repeat_vector (RepeatVector  (None, 150, 64)          0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 150, 64)           33024     
                                                                 
 time_distributed (TimeDistr  (None, 150, 13)          845       
 ibuted)                                                     

In [ ]:
# enc_dec_model.compile(loss      =  'categorical_crossentropy',
#                   optimizer =  'adam',
#                   metrics   =  ['accuracy'])

In [ ]:
enc_dec_training = enc_dec_model.fit(X_train, Y_train, batch_size=256, epochs=5)

Epoch 1/5
180/180 [==============================] - 275s 1s/step - loss: 0.7237 - accuracy: 0.8603
Epoch 2/5
180/180 [==============================] - 271s 2s/step - loss: 0.4692 - accuracy: 0.8717
Epoch 3/5
180/180 [==============================] - 272s 2s/step - loss: 0.3690 - accuracy: 0.8846
Epoch 4/5
180/180 [==============================] - 270s 2s/step - loss: 0.3448 - accuracy: 0.8884
Epoch 5/5
180/180 [==============================] - 271s 2s/step - loss: 0.3331 - accuracy: 0.8910


In [ ]:
print("Model Accuracy on test set = ", enc_dec_model.evaluate(X_test,Y_test))

359/359 [==============================] - 37s 97ms/step - loss: 0.3275 - accuracy: 0.8911
Model Accuracy on test set =  [0.327500581741333, 0.8910742998123169]
